In [ ]:
# 6.15.C.01

# Aufgabe: Überanpassung, Backpropagation
# Die Aufgabe ist es, ein tiefes neuronales Netz zu entwickeln, das die Wahrscheinlichkeit 
#     eines Unfalls anhand von verschiedenen Sensordaten eines Fahrzeugs vorhersagt.

import numpy as np

np.random.seed(42)  # Für reproduzierbare Ergebnisse

# DATEN -> 1000 Messungen mit Zufallsfunktion simuliert - aber in realistischen Bereichen

#    Geschwindigkeit: 0 bis 200 km/h 
geschwindigkeit = np.random.uniform(0, 200, 1000) 
#    Beschleunigung: -3 m/s² (Bremsen) bis 3 m/s² (Beschleunigen) 
beschleunigung = np.random.uniform(-3, 3, 1000) 
#    Bremsverhalten: 0 (nicht bremsend) oder 1 (bremsend) 
bremsverhalten = np.random.randint(0, 2, 1000) 
#    Lenkwinkel: -45 Grad (links) bis 45 Grad (rechts) 
lenkwinkel = np.random.uniform(-45, 45, 1000) 

# MATRIX -> Zusammenführen der Daten in eine Matrix 

daten_matrix = np.column_stack((geschwindigkeit, beschleunigung, bremsverhalten, lenkwinkel)) 

# DATEN VORBEREITUNG -> geplante Normalisieren und Aufteilen, Labels

#    Normalisieren der Sensordaten mithilfe der Min-Max-Normalisierung
min_vals = np.min(daten_matrix, axis=0)
max_vals = np.max(daten_matrix, axis=0)
daten_matrix = (daten_matrix - min_vals) / (max_vals - min_vals)

#    Labels generieren (Unfallwahrscheinlichkeit basierend auf bestimmten Bedingungen)
#           Labels, bei denen die Unfallwahrscheinlichkeit 1 ist, wenn die Geschwindigkeit größer als 100 km/h ist und das Bremsverhalten aktiv ist (d.h. bremsverhalten == 1)
y_labels = ((geschwindigkeit > 100) & (bremsverhalten == 1)).astype(int).reshape(-1, 1)

#    Aufteilen der Daten in 80% Trainings- und 20% Testdatensatz
train_size = int(0.8 * len(daten_matrix))
X_train, X_test = daten_matrix[:train_size], daten_matrix[train_size:]
y_train, y_test = y_labels[:train_size], y_labels[train_size:]

# BACKPROPAGATION

class NeuronalesNetz:
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, lr=0.01, l2_lambda=0.0001):
        self.lr = lr
        self.l2_lambda = l2_lambda
        
        # Initialisierung der Gewichte und Biases für jede Schicht
        self.W1 = np.random.randn(input_size, hidden_size1) * 0.01
        self.b1 = np.zeros((1, hidden_size1))
        self.W2 = np.random.randn(hidden_size1, hidden_size2) * 0.01
        self.b2 = np.zeros((1, hidden_size2))
        self.W3 = np.random.randn(hidden_size2, output_size) * 0.01
        self.b3 = np.zeros((1, output_size))
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return (x > 0).astype(float)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    ''' 
    # diese funktion ist hier nicht benötigt !
    def sigmoid_derivative(self, x):  # ??????????????
        return x * (1 - x)
    '''
    
    def feedforward(self, X):
        # # Vorwärtsdurchlauf (Feedforward)
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)    # Erste versteckte Schicht
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.relu(self.Z2)    # Zweite versteckte Schicht
        self.Z3 = np.dot(self.A2, self.W3) + self.b3
        self.A3 = self.sigmoid(self.Z3)  # Ausgabeschicht 
        return self.A3
    
    def backpropagation(self, X, y):
        m = X.shape[0]
        output = self.feedforward(X)
        error = output - y
        
        # Fehler und Gradientenberechnung für jede Schicht
        dZ3 = error * (output * (1 - output))
        dW3 = np.dot(self.A2.T, dZ3) / m + self.l2_lambda * self.W3
        db3 = np.sum(dZ3, axis=0, keepdims=True) / m
        
        dZ2 = np.dot(dZ3, self.W3.T) * self.relu_derivative(self.A2)
        dW2 = np.dot(self.A1.T, dZ2) / m + self.l2_lambda * self.W2
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m
        
        dZ1 = np.dot(dZ2, self.W2.T) * self.relu_derivative(self.A1)
        dW1 = np.dot(X.T, dZ1) / m + self.l2_lambda * self.W1
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m
        
        # Update der Gewichte
        self.W3 -= self.lr * dW3
        self.b3 -= self.lr * db3
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2 
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
    
    def train(self, X, y, epochs=1000):
        for i in range(epochs):
            self.backpropagation(X, y)
            if i % 100 == 0:
                loss = np.mean((self.feedforward(X) - y) ** 2)
                print(f'Epoch {i}, Loss: {loss:.4f}')
    
    def accuracy(self, X, y):
        predictions = self.feedforward(X) >= 0.5
        return np.mean(predictions == y)

# ARCHITEKTUR -> geplante Schichten & Neuronen des tiefen neuronalen Netzes

#    Eingabeschicht:        4 Neuronen (für jede Sensordatenkategorie eines)
input_size = 4 
#    Versteckte Schicht 1: 64 Neuronen, Aktivierungsfunktion: ReLU 
hidden_size1 = 64
#    Versteckte Schicht 2: 32 Neuronen, Aktivierungsfunktion: ReLU 
hidden_size2 = 32
#    Ausgabeschicht:        1 Neuron,   Aktivierungsfunktion: Sigmoid (für die Unfallwahrscheinlichkeit)
output_size = 1

# MODELL -> Modell erstellen und trainieren

#    Modell erstellen
netz = NeuronalesNetz(input_size, hidden_size1, hidden_size2, output_size, lr=0.01, l2_lambda=0.0001)
#    Training
netz.train(X_train, y_train, epochs=1000)

# ERGEBNISSE -> Ausgabe

print(f'Trainingsgenauigkeit: {netz.accuracy(X_train, y_train) * 100:.2f}%')
print(f'Testgenauigkeit: {netz.accuracy(X_test, y_test) * 100:.2f}%')

# ERKLÄRUNG -> Antwort auf die Frage "Fehlerattributierung & Überanpassung"
#
# Frage:
# Erkläre, wie du die Fehlerattribuierung durchführst, um das Netzwerk vor Überanpassung 
# zu schützen und indirekte Korrelationen zu lernen. 
# 
# Antwort:
# Das neuronale Netz verwendet Backpropagation, um Fehler zu attribuieren und Gewichte anzupassen.
# Der Hauptmechanismus zur Vermeidung von Überanpassung (Overfitting) sind:
#     1) L2-Regularisierung (Gewichtsstrafe): 
#        Bei der Berechnung der Gradienten wird eine zusätzliche Komponente hinzugefügt: + self.l2_lambda * W
#        Dadurch werden große Gewichtswerte bestraft und das Netz bleibt „glatt“.
#     2) sigmoid- und ReLU-Aktivierungsfunktionen, um den Gradientenfluss zu steuern.
#     3) Indirekte Korrelationen durch tiefe Schichten:
#        2 versteckten Schichten (64 und 32 Neuronen) lernen nichtlineare Kombinationen der Eingangsdaten.
#        ReLU hilft dabei, nicht-lineare Zusammenhänge zu erfassen.
#        Sigmoid-Aktivierung in der Ausgabeschicht zwingt das Modell, eine Wahrscheinlichkeitsverteilung 
#             für Unfallrisiken zu lernen.
#     4) Verlust (Loss) wird nach 100 Epochen ausgegeben, um die Netzwerkanpassung zu überwachen.
#     5) Nach dem Training wird die Testgenauigkeit berechnet:
#        Falls sie stark von der Trainingsgenauigkeit abweicht, liegt Overfitting vor.
# Zusammenfassung:
#     Die Kombination aus L2-Regularisierung, Aktivierungsfunktionen, tiefen Schichten und 
#     iterativem Verlust-Tracking ermöglicht es dem Modell, indirekte Korrelationen zu lernen,
#     ohne zu sehr an die Trainingsdaten angepasst zu werden. 

Epoch 0, Loss: 0.2499
Epoch 100, Loss: 0.2419
Epoch 200, Loss: 0.2349
Epoch 300, Loss: 0.2287
Epoch 400, Loss: 0.2233
Epoch 500, Loss: 0.2185
Epoch 600, Loss: 0.2144
Epoch 700, Loss: 0.2108
Epoch 800, Loss: 0.2076
Epoch 900, Loss: 0.2048
Trainingsgenauigkeit: 76.12%
Testgenauigkeit: 77.50%
